In [1]:
import requests
import joblib
import re
import pandas as pd

In [43]:
import pandas as pd
import requests
import joblib

# 사용자 입력 데이터 (예제)
user_input = {
    "year": 2025,
    "month": 1,
    "day": 21,
    "airline": "대한항공",
    "flight_number": "KE703",
    "departure_time": "21:53"
}

# 항공사 및 편명 숫자로 변환
def parse_flight_info(airline, flight_number):
    """항공사 & 편명을 숫자로 변환 (새로운 값이면 추가)"""
    use_data = pd.read_csv('../data/merged_data/data_merge/merged_data.csv')  # 준비된 데이터
    
    # 항공사 및 편명 매핑 테이블 생성
    airline_mapping = {name: code for code, name in enumerate(use_data['항공사'].astype('category').cat.categories)}
    flight_mapping = {name: code for code, name in enumerate(use_data['편명'].astype('category').cat.categories)}
    
    if airline not in airline_mapping:
        airline_mapping[airline] = max(airline_mapping.values(), default=0) + 1
    if flight_number not in flight_mapping:
        flight_mapping[flight_number] = max(flight_mapping.values(), default=0) + 1
    
    return airline_mapping[airline], flight_mapping[flight_number]

# 시간 변환 (YYYYMMDDHH00)
def format_datetime_for_api_fixed_minutes(year, month, day, departure_time):
    hour, _ = map(int, departure_time.split(":"))  # 분은 00으로 설정
    return f"{year}{month:02d}{day:02d}{hour:02d}00", hour

# 기상 정보 API 호출
def get_weather_info(year, month, day, departure_time):
    """API를 통해 특정 시간대의 풍향, 풍속, 기온, 기압 데이터를 가져옴"""
    fctm, hour = format_datetime_for_api_fixed_minutes(year, month, day, departure_time)
    
    url = 'http://apis.data.go.kr/1360000/AirInfoService/getAirInfo'
    params = {
        'numOfRows': '100',
        'pageNo': '1',
        'dataType': 'JSON',
        'fctm': fctm,  # 변환된 날짜 및 시간 (YYYYMMDDHH00)
        'icaoCode': 'RKSI'  # 인천공항 (변경 가능)
    }

    response = requests.get(url, params=params)
    
    try:
        data = response.json()
        if 'response' in data and 'body' in data['response']:
            items = data['response']['body']['items']['item']
            for item in items:
                if item['tmFc'] == fctm:  # 정확한 시간 데이터 찾기
                    return item['wd'], item['ws'], item['ta'], item['qnh'], hour

    except Exception as e:
        print("API 호출 중 오류 발생:", e)

    return None, None, None, None, hour  # 기상 정보 못 찾으면 None 반환

# 사용자 입력 변환 함수
def transform_user_input(user_input, model):
    """사용자 입력을 모델 입력값으로 변환"""
    year, month, day = user_input["year"], user_input["month"], user_input["day"]
    airline, flight_number = user_input["airline"], user_input["flight_number"]
    departure_time = user_input["departure_time"]

    # 항공사 및 편명 숫자로 변환
    airline_id, flight_id = parse_flight_info(airline, flight_number)

    # 시간 변환
    fctm, hour = format_datetime_for_api_fixed_minutes(year, month, day, departure_time)
    minutes = hour * 60 + int(departure_time.split(":")[1])  # 시간(분) 변환

    # 시간대 변환 (0: 0~6시, 1: 6~12시, 2: 12~18시, 3: 18~24시)
    time_slot = pd.cut([hour], bins=[0, 6, 12, 18, 24], labels=[0, 1, 2, 3], right=False)[0]

    # 요일 & 계절 변환
    weekday = pd.Timestamp(year, month, day).dayofweek  # 0: 월요일, 6: 일요일
    season = (month % 12) // 3  # 0: 겨울, 1: 봄, 2: 여름, 3: 가을

    # 기상 정보 가져오기
    wind_dir, wind_speed, temp, pressure, hour = get_weather_info(year, month, day, departure_time)

    # 변환된 데이터 반환
    transformed_data = pd.DataFrame([[
        airline_id, flight_id, int(wind_dir) if wind_dir else 180, int(wind_speed) if wind_speed else 5,
        int(temp) if temp else 20, int(pressure) if pressure else 1013, weekday, season, hour,
        int(time_slot), year, month, day, minutes
    ]], columns=model.feature_names_in_)

    return transformed_data

# 모델 예측 함수
def predict_flight_status(user_input, model_path="../ML/with_undersampling/trained_model/random_forest_undersampled_model.pkl"):
    model = joblib.load(model_path)
    transformed_input = transform_user_input(user_input, model)
    probabilities = model.predict_proba(transformed_input)[0]
    status_mapping = {0: "출발", 1: "지연", 2: "취소", 3: "회항"}
    
    result = {status_mapping[i]: round(probabilities[i] * 100, 2) for i in range(len(probabilities))}
    return result

# 실행 및 예측 확률 출력
prediction_result = predict_flight_status(user_input)
print("예측 확률:", prediction_result)

예측 확률: {'출발': np.float64(43.0), '지연': np.float64(48.0), '취소': np.float64(9.0), '회항': np.float64(0.0)}
